In [1]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
#from torch.optim.lr_scheduler import _LRScheduler

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torchvision import models
from torchsummary import summary
import torch.nn.functional as F


In [2]:

# GPU 사용이 가능할 경우, GPU를 사용할 수 있게 함.'
#os.environ['CUDA_VISIBLE_DEVICES'] = '3'
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(device)

#print(os.environ.get('CUDA_VISIBLE_DEVICES'))

cuda


In [3]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# 클래스별 IoU를 계산하기 위한 함수
def calculate_iou_per_class(y_true, y_pred, class_id):
    intersection = np.sum((y_true == class_id) & (y_pred == class_id))
    union = np.sum((y_true == class_id) | (y_pred == class_id))
    iou = intersection / union if union > 0 else 0
    return iou

In [4]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        #directory_path = "/mnt/nas27/Dataset/Samsung_DM"
        directory_path = './data/224'
        img_path = self.data.iloc[idx, 1]
        img_path = os.path.join(directory_path, img_path)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        
        mask_path = self.data.iloc[idx, 2]
        mask_path = os.path.join(directory_path, mask_path)
        mask = cv2.imread(mask_path)
        #mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask = np.round(mask).astype(np.uint8)
        mask[mask > 12] = 12 #배경을 픽셀값 12로 간주

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

# class CustomDataset_target(Dataset):
#     def __init__(self, csv_file, transform=None, infer=False):
#         self.data = pd.read_csv(csv_file)
#         self.transform = transform
#         self.infer = infer

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         directory_path = "/mnt/nas27/Dataset/Samsung_DM"
#         img_path = self.data.iloc[idx, 1]
#         img_path = os.path.join(directory_path, img_path[2:])
#         image = cv2.imread(img_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
#         if self.infer:
#             if self.transform:
#                 image = self.transform(image=image)['image']
#             return image


#         if self.transform:
#             augmented = self.transform(image=image)
#             image = augmented['image']
            

#         return image
     

transform = A.Compose(
    [   
        #A.Resize(224, 224),
        #A.Resize(128, 128),
        A.Normalize(),
        
        # 변형
        # A.VerticalFlip(p=0.5),
        # A.RandomRotate90(p=0.5),
        # A.HueSaturationValue(p=0.2),
        
        ToTensorV2()
    ]
)

In [5]:
#Unet의 기본이 되는 conv블럭
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)  # 여기서 in_channels는 out_channels와 동일해야 합니다.
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        return x

#인코더 블럭
class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(EncoderBlock,self).__init__()
        self.convblock1 = ConvBlock(in_channels, out_channels)  # 첫 번째 ConvBlock의 in_channels는 입력 이미지의 채널 수와 일치해야 합니다.
        #self.convblock2 = ConvBlock(out_channels, out_channels)  # 두 번째 ConvBlock의 in_channels는 out_channels와 일치해야 합니다.
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self,x):
        x = self.convblock1(x)
        #x = self.convblock2(x)
        p = self.maxpool(x)
        return x , p
#디코더 블럭
#디코더는 업샘플링 이후 스킵연결과 붙어서 convblock을 통과해야함
#skip보다 작은 x x먼저 업샘플링 32 -> 64 , skip과 결합 6464 
class DecoderBlock(nn.Module):
    def __init__(self, channels):
        super(DecoderBlock,self).__init__()
        self.upsample = nn.ConvTranspose2d(channels*2, channels, kernel_size=4, stride=2, padding=1)#x 업샘플링
        self.convblock1 = ConvBlock(channels*2, channels)#차원감소
        #self.convblock2 = ConvBlock(channels, channels)
    def forward(self,x,skip):
        x = self.upsample(x)
        x = torch.cat([x, skip], dim=1)
        x = self.convblock1(x)
        #x = self.convblock2(x)
        return x

###########################################
class GradReverse(torch.autograd.Function):
    def forward(self, x):
        return x.view_as(x)

    def backward(self, grad_output): # 역전파 시에 gradient에 음수를 취함
        return (grad_output * -1)

class domain_classifier(nn.Module):
    def __init__(self):
        super(domain_classifier, self).__init__()
        self.fc1 = nn.Linear(224*224*64, 10)
        self.fc2 = nn.Linear(10, 1) # source = 0, target = 1 회귀 가정

    def forward(self, x):
        x = x.view(-1, 224*224*64)
        x = GradReverse.apply(x) # gradient reverse
        x = F.leaky_relu(self.fc1(x))
        x = self.fc2(x)
        
        return torch.sigmoid(x)

###########################################


#Unet구조 middle의 xm값의 움직임에 주의
class Unet(nn.Module):
    def __init__(self,n_classes):
        super(Unet,self).__init__()
        self.encoder1 = EncoderBlock(3,64)
        self.encoder2 = EncoderBlock(64,128)
        self.encoder3 = EncoderBlock(128,256)
        self.encoder4 = EncoderBlock(256,512)
        
        self.middleconv = ConvBlock(512,1024)
        
        
        self.decoder4 = DecoderBlock(512)
        self.decoder3 = DecoderBlock(256)
        self.decoder2 = DecoderBlock(128)
        self.decoder1 = DecoderBlock(64)
        self.segmap = nn.Conv2d(64,n_classes, kernel_size=1)
        
        self.domain_classifier = domain_classifier()
                                        

    def forward(self,x):
        x1,p = self.encoder1(x)#3->64   #P:256,256 x1 :512,512
        x2,p = self.encoder2(p)#64->128 #P:128,128 x2:256,256
        x3,p = self.encoder3(p)#128->256#p:64,64 x3:128,128
        x4,p = self.encoder4(p)#256->512#p:32,32 x4:64,64
        
        xm = self.middleconv(p)#512->1024#32,32
        
        x = self.decoder4(xm,x4)#뉴런:1024->512->512 #출력tensor:64,64
        x = self.decoder3(x,x3)#뉴런:512->256->256 #출력tensor:128,128
        x = self.decoder2(x,x2)#뉴런:256->128->128 #출력tensor:256,256
        x = self.decoder1(x,x1)#뉴런:128->64->64 #출력tensor:512,512

        x_c = self.segmap(x)
        x_d = self.domain_classifier(x)

        return x_c, x_d

In [6]:
import pickle

# 저장된 class_weights를 불러옵니다.
class_weights_path = 'CLASS_WEIGHTS.pkl'

with open(class_weights_path, 'rb') as file:
    CLASS_WEIGHTS = pickle.load(file)

print(CLASS_WEIGHTS)

tensor([   6.2232,   81.0473,    5.1017,   66.2180,  170.2419,  506.2671,
         159.9452,    6.9654,    4.3830,  907.1918, 2223.4863,   10.2576,
           7.6339], device='cuda:0')


## Loss Function

In [7]:
# loss function과 optimizer 정의

class DANN_Loss(nn.Module):
    def __init__(self):
        super(DANN_Loss, self).__init__()

        self.CE = nn.CrossEntropyLoss(weight=CLASS_WEIGHTS) # 0~9 class 분류용
        self.BCE = nn.BCELoss() # 도메인 분류용
        
    # result : DANN_CNN에서 반환된 값
    # label : 숫자 0 ~ 9에 대한 라벨
    # domain_num : 0(source) or 1(target)
    def forward(self, result, label, domain_num, alpha = 1):
        label_logits, domain_logits = result # DANN_CNN의 결과

        batch_size = domain_logits.shape[0]

        domain_target = torch.FloatTensor([domain_num] * batch_size).unsqueeze(1).to(device)

        domain_loss = self.BCE(domain_logits, domain_target) # domain 분류 loss
        target_loss = self.CE(label_logits, label) # class 분류 loss

        loss = target_loss + alpha * domain_loss

        return loss
    

loss_fn = DANN_Loss().to(device)

#criterion =nn.CrossEntropyLoss()
#domain_criterion = nn.BCELoss()
#criterion = nn.CrossEntropyLoss(weight=class_weights)

In [8]:
LR = 0.001
EP = 10
BATCH_SIZE = 8
ACCMULATION_STEP = 1 
N_CLASSES = 13 #IoU 점수측정하기 위한 클래스의 개수
ALPHA = 0.5
# model 초기화
#model = Unet_resnet18(n_classes = N_CLASSES).to(device)
#model = ResNet50(num_classes=N_CLASSES).to(device)
model = Unet(n_classes = N_CLASSES).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

optimizer.zero_grad() 

source_dataset = CustomDataset(csv_file='./data/DA_csv/100train_source.csv', transform=transform)
source_dataloader = DataLoader(source_dataset, batch_size=BATCH_SIZE, shuffle=True)
target_dataset = CustomDataset(csv_file='./data/DA_csv/100fish_train_source.csv', transform=transform)
target_dataloader = DataLoader(target_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_source_dataset = CustomDataset(csv_file='./data/DA_csv/20val_source.csv', transform=transform)
val_source_dataloader = DataLoader(val_source_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_target_dataset = CustomDataset(csv_file='./data/DA_csv/20fish_val_source.csv', transform=transform)
val_target_dataloader = DataLoader(val_target_dataset, batch_size=BATCH_SIZE, shuffle=False)
# source_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./train_source.csv'), transform=transform)#
# source_dataloader = DataLoader(source_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
# s_valid_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./val_source.csv'), transform=transform)
# s_valid_dataloader = DataLoader(s_valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)
# #target_dataset = CustomDataset_target(csv_file='./f_train_source.csv', transform=transform)
# target_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./fish_val_source.csv'), transform=transform)
# target_dataloader = DataLoader(target_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)

# t_test_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./test.csv'), transform=transform, infer=True)
# t_test_dataloader = DataLoader(t_test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)

In [9]:
# import pandas as pd
# import os
# import cv2
# import matplotlib.pyplot as plt
# a1 = pd.read_csv("./data/6_fish_source.csv")
# p = "./data/224/"
# a2 = os.path.join(p, a1.iloc[3,2])
# a3 = cv2.imread(a2)
# a4 = cv2.cvtColor(a3, cv2.COLOR_BGR2GRAY)
# a4 = np.round(a4).astype(np.uint8)
# a5 = a4*20

# plt.imshow(a5, cmap='gray')
# plt.axis('off')
# plt.show()

In [10]:
import random
#torch.cuda.empty_cache()
#import wandb


# wandb.init(
#     # set the wandb project where this run will be logged
#     project="practice_10_06",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": LR,
#     "architecture": "CNN",
#     "dataset": "Samsung",
#     "epochs": EP,
#     }
# )

for epoch in range(EP):
    # 클래스별 IoU를 누적할 리스트 초기화
    train_class_ious = []
    fish_train_class_ious = []
    # 학습
    model.train()
    epoch_loss = 0

    for source_data, target_data in tqdm(zip(source_dataloader, target_dataloader)):

        source_images = source_data[0].float().to(device)
        source_masks = source_data[1].long().to(device)
        target_images = target_data[0].float().to(device)
        target_masks = target_data[1].long().to(device)

        optimizer.zero_grad()
        source_outputs = model(source_images)
        target_outputs = model(target_images)

        source_loss = loss_fn(source_outputs, source_masks, 0, alpha = ALPHA)
        target_loss = loss_fn(target_outputs, target_masks, 1, alpha = ALPHA)

        loss = source_loss + target_loss

        epoch_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #scheduler.step()
                # train 클래스별 IoU 계산
        source_outputs = torch.softmax(source_outputs[0], dim=1).cpu()
        source_outputs = torch.argmax(source_outputs, dim=1).numpy()

        for class_id in range(N_CLASSES):
            iou = calculate_iou_per_class(np.array(source_masks.cpu()), np.array(source_outputs), class_id)
            train_class_ious.append(iou)

        target_outputs = torch.softmax(target_outputs[0], dim=1).cpu()
        target_outputs = torch.argmax(target_outputs, dim=1).numpy()

        for class_id in range(N_CLASSES):
            iou = calculate_iou_per_class(np.array(target_masks.cpu()), np.array(target_outputs), class_id)
            fish_train_class_ious.append(iou)

    train_class_ious = np.array(train_class_ious).reshape(N_CLASSES, -1)
    train_class_ious = np.mean(train_class_ious, axis=1)
    print("--IoU Scores Train--")
    for class_id, iou in enumerate(train_class_ious):
        print(f'Class{class_id}: {iou:.4f}', end=" ")
        if (class_id+1) % 7 == 0:
            print()

    fish_train_class_ious = np.array(fish_train_class_ious).reshape(N_CLASSES, -1)
    fish_train_class_ious = np.mean(fish_train_class_ious, axis=1)
    print()
    print("--IoU Scores Fish Train--")
    for class_id, iou in enumerate(fish_train_class_ious):
        print(f'Class{class_id}: {iou:.4f}', end=" ")
        if (class_id+1) % 7 == 0:
            print()

    # mIoU 계산
    train_mIoU = np.mean(train_class_ious)
    fish_train_mIoU = np.mean(fish_train_class_ious)

    # 에폭마다 결과 출력 
    print(f"\nEpoch{epoch+1}")
    print(f"Train Loss: {(loss/len(source_dataloader))}")
    print(f"Train mIoU: {train_mIoU}, Fish_Train_mIoU: {fish_train_mIoU}" )
    print("___________________________________________________________________________________________\n")


    #################################################################
    # # 클래스별 IoU를 누적할 리스트 초기화
    # val_class_ious = []
    # fish_val_class_ious = []
    # epoch_loss = 0
    # # 학습
    # with torch.no_grad():
    #     model.eval()

    #     for source_data, target_data in tqdm(zip(val_source_dataloader, val_target_dataloader)):

    #         source_images = source_data[0].float().to(device)
    #         source_masks = source_data[1].long().to(device)
    #         target_images = target_data[0].float().to(device)
    #         target_masks = target_data[1].long().to(device)

    #         source_outputs = model(source_images)
    #         target_outputs = model(target_images)

    #         source_loss = loss_fn(source_outputs, source_masks, 0, alpha = ALPHA)
    #         target_loss = loss_fn(target_outputs, target_masks, 1, alpha = ALPHA)

    #         loss = source_loss + target_loss

    #         epoch_loss += loss.item()

    #         # train 클래스별 IoU 계산
    #         source_outputs = torch.softmax(source_outputs[0], dim=1).cpu()
    #         source_outputs = torch.argmax(source_outputs, dim=1).numpy()

    #         for class_id in range(N_CLASSES):
    #             iou = calculate_iou_per_class(np.array(source_masks.cpu()), np.array(source_outputs), class_id)
    #             val_class_ious.append(iou)

    #         target_outputs = torch.softmax(target_outputs[0], dim=1).cpu()
    #         target_outputs = torch.argmax(target_outputs, dim=1).numpy()

    #         for class_id in range(N_CLASSES):
    #             iou = calculate_iou_per_class(np.array(target_masks.cpu()), np.array(target_outputs), class_id)
    #             fish_val_class_ious.append(iou)

    # val_class_ious = np.array(val_class_ious).reshape(N_CLASSES, -1)
    # val_class_ious = np.mean(val_class_ious, axis=1)
    # print("--IoU Scores val--")
    # for class_id, iou in enumerate(val_class_ious):
    #     print(f'Class{class_id}: {iou:.4f}', end=" ")
    #     if (class_id+1) % 7 == 0:
    #         print()

    # fish_val_class_ious = np.array(fish_val_class_ious).reshape(N_CLASSES, -1)
    # fish_val_class_ious = np.mean(fish_val_class_ious, axis=1)
    # print()
    # print("--IoU Scores Fish val--")
    # for class_id, iou in enumerate(fish_val_class_ious):
    #     print(f'Class{class_id}: {iou:.4f}', end=" ")
    #     if (class_id+1) % 7 == 0:
    #         print()

    # # mIoU 계산
    # val_mIoU = np.mean(val_class_ious)
    # fish_val_mIoU = np.mean(fish_val_class_ious)

    # # 에폭마다 결과 출력 
    # print(f"\nEpoch{epoch+1}")
    # print(f"Train Loss: {(loss/len(source_dataloader))}")
    # print(f"Train mIoU: {val_mIoU}, Fish_Train_mIoU: {fish_val_mIoU}" )
    # print("___________________________________________________________________________________________\n")

#     # log metrics to wandb
#     wandb.log({"train score": train_mIoU, "train loss": epoch_loss})
#     wandb.log({"val score": val_mIoU, "val loss": val_loss})
    
    
# # [optional] finish the wandb run, necessary in notebooks
# wandb.finish()


0it [00:00, ?it/s]

13it [01:42,  7.87s/it]


--IoU Scores Train--
Class0: 0.0114 Class1: 0.0338 Class2: 0.0779 Class3: 0.0498 Class4: 0.0396 Class5: 0.0412 Class6: 0.0227 
Class7: 0.0349 Class8: 0.0385 Class9: 0.0366 Class10: 0.0296 Class11: 0.0327 Class12: 0.0267 
--IoU Scores Fish Train--
Class0: 0.0077 Class1: 0.0087 Class2: 0.0472 Class3: 0.0350 Class4: 0.0365 Class5: 0.0347 Class6: 0.0353 
Class7: 0.0355 Class8: 0.0353 Class9: 0.0307 Class10: 0.0305 Class11: 0.0315 Class12: 0.0453 
Epoch1
Train Loss: 4.222572326660156
Train mIoU: 0.036571221800543015, Fish_Train_mIoU: 0.03182807385356154
___________________________________________________________________________________________



13it [01:45,  8.14s/it]


--IoU Scores Train--
Class0: 0.0327 Class1: 0.0230 Class2: 0.0292 Class3: 0.0286 Class4: 0.0223 Class5: 0.0249 Class6: 0.0227 
Class7: 0.0288 Class8: 0.0234 Class9: 0.0260 Class10: 0.0218 Class11: 0.0263 Class12: 0.0191 
--IoU Scores Fish Train--
Class0: 0.0604 Class1: 0.0701 Class2: 0.0714 Class3: 0.0732 Class4: 0.0713 Class5: 0.0712 Class6: 0.0737 
Class7: 0.0737 Class8: 0.0724 Class9: 0.0703 Class10: 0.0722 Class11: 0.0714 Class12: 0.0697 
Epoch2
Train Loss: 4.195852756500244
Train mIoU: 0.025291251363917414, Fish_Train_mIoU: 0.07085061673573549
___________________________________________________________________________________________



...

--IoU Scores Train--
Class0: 0.0570 Class1: 0.0594 Class2: 0.0593 Class3: 0.0567 Class4: 0.0655 Class5: 0.0525 Class6: 0.0717 
Class7: 0.0640 Class8: 0.0616 Class9: 0.0758 Class10: 0.0633 Class11: 0.0770 Class12: 0.0773 
--IoU Scores Fish Train--
Class0: 0.1078 Class1: 0.1057 Class2: 0.1153 Class3: 0.1139 Class4: 0.1078 Class5: 0.1196 Class6: 0.1095 
Class7: 0.1055 Class8: 0.1083 Class9: 0.1057 Class10: 0.1065 Class11: 0.1079 Class12: 0.1075 
Epoch7
Train Loss: 4.140923976898193
Train mIoU: 0.06471417120224868, Fish_Train_mIoU: 0.10929805720239233
___________________________________________________________________________________________



13it [01:47,  8.28s/it]


--IoU Scores Train--
Class0: 0.0538 Class1: 0.0643 Class2: 0.0660 Class3: 0.0803 Class4: 0.0706 Class5: 0.0863 Class6: 0.0865 
Class7: 0.0783 Class8: 0.0691 Class9: 0.0842 Class10: 0.0716 Class11: 0.0819 Class12: 0.0469 
--IoU Scores Fish Train--
Class0: 0.1071 Class1: 0.1038 Class2: 0.1065 Class3: 0.1005 Class4: 0.1075 Class5: 0.1063 Class6: 0.1192 
Class7: 0.1097 Class8: 0.1179 Class9: 0.1107 Class10: 0.1142 Class11: 0.1138 Class12: 0.1023 
Epoch8
Train Loss: 4.143531322479248
Train mIoU: 0.07227833899404218, Fish_Train_mIoU: 0.10919030449187006
___________________________________________________________________________________________



1it [00:11, 11.47s/it]


KeyboardInterrupt: 

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'